<div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 30px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
  Deep Learning for NLP
  </div> 
  
<div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 30px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
  Part I - 1 <br><br><br>
  Word Embedding
  </div> 

  <div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 20px; 
      text-align: center; 
      padding: 15px;">
  </div> 

  <div style=" float:right; 
      font-size: 12px; 
      line-height: 12px; 
  padding: 10px 15px 8px;">
  Jean-baptiste AUJOGUE
  </div> 

### Part I
1. <font color=red>**Word Embedding**</font>

2. Sentence Classification

    _Applications :_
    
    - Extractive Summarization
    - Sentiment Analysis
    - Text segmentation


3. Language Modeling

4. Sentence tagging

    _Applications :_
    
    - Part-of-speech Tagging
    - Named Entity Recognition
    - Automatic Value Extraction
    


### Part II

5. Auto-Encoding

6. Machine Translation

7. Text Classification




### Part III

8. Abstractive Summarization

9. Question Answering

10. Chatbot


</div>

***

<a id="plan"></a>

# Overview

The global purpose of Word Embedding is to map a word, as raw string, to a dense vector. Three approaches are commonly followed for such task :

| Level |  | |
|------|------|------|
| **Word** | [I.1 Custom model](#word_level_custom) | [I.2 Gensim Model](#gensim) |
| **Subword** | [II.1 FastText model](#fastText) |  |
| **Character** |  |  |


<br>
Visualization with TensorBoard : https://www.tensorflow.org/guide/embedding (TODO)

# Training objectives

#### CBOW training objective

Cette méthode de vectorisation est introduite dans \cite{mikolov2013distributed, mikolov2013efficient}, et consiste à construire pour un vocabulaire de mots une table de vectorisation $T$ contenant un vecteur par mot. La spécificité de cette méthode est que cette vectorisation est faite de façon à pouvoir prédire chaque mot à partir de son contexte. La construction de cette table $T$ passe par la création d'un réseau de neurones, qui sert de modèle pour l'estimation de la probabilité de prédiction d'un mot $w_t$ d'après son contexte $c = w_{t-N}, \, ... \, , w_{t-1}$, $w_{t+1}, \, ... \, , w_{t+N}$. La table $T$ intégrée au modèle sera optimisée lorsque ce modèle sera entrainé de façon à ce qu'un mot $w_t$ maximise la vraisemblance de la probabilité $P(. \, | \, c)$ fournie par le modèle. 

Le réseau de neurones de décrit de la façon suivante :

![cbow](figs/CBOW.png)

Un contexte $c = w_{t-N}, \, ... \, , w_{t-1}$, $w_{t+1}, \, ... \, , w_{t+N}$ est vectorisé via une table $T$ fournissant un ensemble de vecteurs denses (typiquement de dimension comprise entre 50 et 300) $T(w_{t-N}), \, ... \, , T(w_{t-1})$, $T(w_{t+1}), \, ... \, , T(w_{t+N})$. Chaque vecteur est ensuite transformé via une transformation affine, dont les vecteurs résultants sont superposés en un unique vecteur

\begin{align*}
v_c = \sum _{i = - N}^N M_i T(w_{t+i}) + b_i
\end{align*}

Le vecteur $v_c$ est de dimension typiquement égale à la dimension de la vectorisation de mots. Une autre table $T'$ est utilisée pour une nouvelle vectorisation du vocabulaire, de sorte que le mot $w_{t}$ soit transformé en un vecteur $T'(w_{t})$ par cette table, et soit proposé en position $t$ avec probabilité

\begin{align*}
P(w_{t} \, | \, c\,) = \frac{\exp\left( T'(w_{t}) \cdot v_c \right) }{\displaystyle \sum _{w \in \mathcal{V}} \exp\left(   T'(w) \cdot v_c 
\right) }
\end{align*}

Ici $\cdot$ désigne le produit scalaire entre vecteurs. L'optimisation de ce modèle permet d'ajuster la table $T$ afin que les vecteurs de mots portent suffisamment d'information pour reformer un mot à partir du contexte.


#### Skip-Gram training objective


Cette méthode de vectorisation est introduite dans \cite{mikolov2013distributed, mikolov2013efficient} comme version mirroir au Continuous Bag Of Words, et consiste là encore à construire pour un vocabulaire de mots une table de vectorisation $T$ contenant un vecteur par mot. La spécificité de cette méthode est que cette vectorisation est faite non pas de façon prédire un mot central $w$ à partir d'un contexte $c $ comme pour CBOW, mais plutôt de prédire le contexte $c $ à partir du mot central $w$. La construction de cette table $T$ passe par la création d'un réseau de neurones servant de modèle pour l'estimation de la probabilité de prédiction d'un contexte $c = w_{t-N}, \, ... \, , w_{t-1}$, $w_{t+1}, \, ... \, , w_{t+N}$ à partir d'un mot central $w_t$. La table $T$ intégrée au modèle sera optimisée lorsque ce modèle sera entrainé de façon à ce que le contexte  $ c $ maximise la vraisemblance de la probabilité $P( . \, | \, w_t)$ fournie par le modèle.


Une implémentation de ce modèle est la suivante : 


![skipgram](figs/Skipgram.png)


Un mot courant $w_t$ est vectorisé par une table $T$ fournissant un vecteur dense (typiquement de dimension comprise entre 50 et 300) $T(w_t)$. Ce vecteur est alors transformé en un ensemble de $2N$ vecteurs

\begin{align*}
\sigma (M_{i} T(w_t) + b_{i}) \qquad \qquad i =-N,\, ...\, , -1, 1, \, ...\, , N
\end{align*}

où $N$ désigne la taille de la fenêtre retenue, d'une dimension typiquement égale à la dimension de la vectorisation de mots, et $\sigma$ une fonction non linéaire (typiquement la _Rectified Linear Unit_ $\sigma (x) = max (0, x)$). Une autre table $T'$ est utilisée pour une nouvelle vectorisation du vocabulaire, de sorte que chaque mot $w_{t+i}$, transformé en un vecteur $T'(w_{t+i})$ par cette table, soit proposé en position $t+i$ avec probabilité

\begin{align*}
P( w_{t+i} | \, w_t) = \frac{\exp\left(  T'(w_{t+i}) ^\perp \sigma \left( M_i T(w_t) + b_{i}\right) \right) }{\displaystyle \sum _{w \in \mathcal{V}} \exp\left(   T'(w) ^\perp \sigma \left( M_i T(w_t) + b_i\right) \right) }
\end{align*}

On modélise alors la probabilité qu'un ensemble de mots $c = w_{t-N}, \, ... \, , w_{t-1}$, $w_{t+1}, \, ... \, , w_{t+N}$ soit le contexte d'un mot $w_t$ par le produit

\begin{align*}
 P( c\, | \, w_t) = \prod _{i = -N}^N P( w_{t+i}\, | \, w_t)
\end{align*}

Ce modèle de probabilité du contexte d'un mot est naif au sens où les mots de contextes sont considérés comme indépendants deux à deux dès lors que le mot central est connu. Cette approximation rend cependant le calcul d'optimisation beaucoup plus court.



L'optimisation de ce modèle permet d'ajuster la table $T$ afin que les vecteurs de mots portent suffisamment d'information pour reformer l'intégralité du contexte à partir de ce seul mot. La vectorisation Skip-Gram est typiquement plus performante que CBOW, car la table $T$ subit plus de contrainte dans son optimisation, et puisque le vecteur d'un mot est obtenu de façon à pouvoir prédire l'utilisation réelle du mot, ici donnée par son contexte. 

# Packages

[Back to top](#plan)

In [12]:
import sys
import warnings
from __future__ import unicode_literals, print_function, division
import os
from io import open
import unicodedata
import string
import time
import math
import re
import random
import pickle
import copy
from unidecode import unidecode


# for special math operation
from sklearn.preprocessing import normalize


# for manipulating data 
import numpy as np
#np.set_printoptions(threshold=np.nan)
import pandas as pd
import bcolz # see https://bcolz.readthedocs.io/en/latest/intro.html
import pickle


# for text processing
import gensim
from gensim.models import KeyedVectors
#import spacy
import nltk
#nltk.download()
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.stem.porter import PorterStemmer


# for deep learning
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


warnings.filterwarnings("ignore")
print('python version :', sys.version)
print('pytorch version :', torch.__version__)
print('DL device :', device)

python version : 3.6.5 |Anaconda, Inc.| (default, Mar 29 2018, 13:32:41) [MSC v.1900 64 bit (AMD64)]
pytorch version : 0.4.0
DL device : cuda


In [13]:
path_to_NLP = 'C:\\Users\\Jb\\Desktop\\NLP'

In [14]:
#sys.path.append(path_to_NLP + '\\chatNLP')

# Corpus

[Back to top](#plan)

Le texte est importé et mis sous forme de liste, où chaque élément représente un texte présenté sous forme d'une liste de mots.<br> Le corpus et donc une fois importé sous le forme : [[str]]

In [15]:
def cleanSentence(sentence): # -------------------------  str
    sw = ['']
    #sw += nltk.corpus.stopwords.words('english')
    #sw += nltk.corpus.stopwords.words('french')

    def unicodeToAscii(s):
        """Turn a Unicode string to plain ASCII, thanks to http://stackoverflow.com/a/518232/2809427"""
        return ''.join( c for c in unicodedata.normalize('NFD', s)
                        if unicodedata.category(c) != 'Mn')

    def normalizeString(s):
        '''Remove rare symbols from a string'''
        s = unicodeToAscii(s.lower().strip()) # 
        #s = re.sub(r"[^a-zA-Z\.\(\)\[\]]+", r" ", s)  # 'r' before a string is for 'raw' # ?&\%\_\- removed # set('''.,:;()*#&-_%!?/\'")''')
        return s

    def wordTokenizerFunction():
        # base version
        function = lambda sentence : sentence.strip().split()

        # nltk version
        #function = word_tokenize    
        return function

    # 1 - caractères spéciaux
    def clean_sentence_punct(text): # --------------  str
        text = normalizeString(text)
        # suppression de la dernière ponctuation
        if (len(text) > 0 and text[-1] in ['.', ',', ';', ':', '!', '?']) : text = text[:-1]

        text = text.replace(r'(', r' ( ')
        text = text.replace(r')', r' ) ')
        text = text.replace(r'[', r' [ ')
        text = text.replace(r']', r' ] ')
        text = text.replace(r'<', r' < ')
        text = text.replace(r'>', r' > ')

        text = text.replace(r':', r' : ')
        text = text.replace(r';', r' ; ')
        for i in range(5) :
            text = re.sub('(?P<val1>[0-9])\.(?P<val2>[0-9])', '\g<val1>__-__\g<val2>', text)
            text = re.sub('(?P<val1>[0-9]),(?P<val2>[0-9])', '\g<val1>__-__\g<val2>', text)
        text = text.replace(r',', ' , ')
        text = text.replace(r'.', ' . ')
        for i in range(5) : text = re.sub('(?P<val1>[p0-9])__-__(?P<val2>[p0-9])', '\g<val1>.\g<val2>', text)
        text = re.sub('(?P<val1>[0-9]) \. p \. (?P<val2>[0-9])', '\g<val1>.p.\g<val2>', text)
        text = re.sub('(?P<val1>[0-9]) \. s \. (?P<val2>[0-9])', '\g<val1>.s.\g<val2>', text)

        text = text.replace(r'"', r' " ')
        text = text.replace(r'’', r" ' ")
        text = text.replace(r'”', r' " ')
        text = text.replace(r'“', r' " ')
        text = text.replace(r'/', r' / ')

        text = re.sub('(…)+', ' … ', text)
        text = text.replace('≤', ' ≤ ')          
        text = text.replace('≥', ' ≥ ')
        text = text.replace('°c', ' °c ')
        text = text.replace('°C', ' °c ')
        text = text.replace('ºc', ' °c ')
        text = text.replace('n°', 'n° ')
        text = text.replace('%', ' % ')
        text = text.replace('*', ' * ')
        text = text.replace('+', ' + ')
        text = text.replace('-', ' - ')
        text = text.replace('_', ' ')
        text = text.replace('®', ' ')
        text = text.replace('™', ' ')
        text = text.replace('±', ' ± ')
        text = text.replace('÷', ' ÷ ')
        text = text.replace('–', ' - ')
        text = text.replace('μg', ' µg')
        text = text.replace('µg', ' µg')
        text = text.replace('µl', ' µl')
        text = text.replace('μl', ' µl')
        text = text.replace('µm', ' µm')
        text = text.replace('μm', ' µm')
        text = text.replace('ppm', ' ppm')
        text = re.sub('(?P<val1>[0-9])mm', '\g<val1> mm', text)
        text = re.sub('(?P<val1>[0-9])g', '\g<val1> g', text)
        text = text.replace('nm', ' nm')

        text = re.sub('fa(?P<val1>[0-9])', 'fa \g<val1>', text)
        text = re.sub('g(?P<val1>[0-9])', 'g \g<val1>', text)
        text = re.sub('n(?P<val1>[0-9])', 'n \g<val1>', text)
        text = re.sub('p(?P<val1>[0-9])', 'p \g<val1>', text)
        text = re.sub('q_(?P<val1>[0-9])', 'q_ \g<val1>', text)
        text = re.sub('u(?P<val1>[0-9])', 'u \g<val1>', text)
        text = re.sub('ud(?P<val1>[0-9])', 'ud \g<val1>', text)
        text = re.sub('ui(?P<val1>[0-9])', 'ui \g<val1>', text)

        text = text.replace('=', ' ')
        text = text.replace('!', ' ')
        text = text.replace('-', ' ')
        text = text.replace(r' , ', ' ')
        text = text.replace(r' . ', ' ')

        text = re.sub('(?P<val>[0-9])ml', '\g<val> ml', text)
        text = re.sub('(?P<val>[0-9])mg', '\g<val> mg', text)

        for i in range(5) : text = re.sub('( [0-9]+ )', ' ', text)
        #text = re.sub('cochran(\S)*', 'cochran ', text)
        return text

    # 3 - split des mots
    def wordSplit(sentence, tokenizeur): # ------------- [str]
        return tokenizeur(sentence)

    # 4 - mise en minuscule et enlèvement des stopwords
    def stopwordsRemoval(sentence, sw): # ------------- [[str]]
        return [word for word in sentence if word not in sw]

    # 6 - correction des mots
    def correction(text):
        def correct(word):
            return spelling.suggest(word)[0]
        list_of_list_of_words = [[correct(word) for word in sentence] for sentence in text]
        return list_of_list_of_words

    # 7 - stemming
    def stemming(text): # ------------------------- [[str]]
        list_of_list_of_words = [[PorterStemmer().stem(word) for word in sentence if word not in sw] for sentence in text]
        return list_of_list_of_words


    tokenizeur = wordTokenizerFunction()
    sentence = clean_sentence_punct(str(sentence))
    sentence = wordSplit(sentence, tokenizeur)
    sentence = stopwordsRemoval(sentence, sw)
    #text = correction(text)
    #text = stemming(text)
    return sentence


def importSheet(file_name) :
    def cleanDatabase(db):
        words = []
        title = ''
        for pair in db :
            #print(pair)
            current_tile = pair[0].split(' | ')[-1]
            if current_tile != title :
                words += cleanSentence(current_tile) #[str]
                title  = current_tile                # str
            words += cleanSentence(str(pair[1]).split(' | ')[-1])     #[str]
        return words

    df = pd.read_excel(file_name, sep = ',', header = None)
    headers = [i for i, titre in enumerate(df.ix[0,:].values) if i in [1, 2] or titre == 'score manuel'] 
    db = df.ix[1:, headers].values.tolist()
    db = [el[:2] for el in db if el[-1] in [0,1, 10]]
    words = cleanDatabase(db)
    return words


def importCorpus(path_to_data) :
    corpus = []
    reps = os.listdir(path_to_data)
    for rep in reps :
        files = os.listdir(path_to_data + '\\' + rep)
        for file in files :
            file_name = path_to_data + '\\' + rep + '\\' + file
            corpus.append(importSheet(file_name))
    return corpus

In [16]:
corpus = importCorpus(path_to_NLP + '\\data\\AMM')

<a id="word_level"></a>


# 1 Word-level Embedding
***

<a id="word_level_custom"></a>

## 1.1 Custom Word-level Embedding Model

[Back to top](#plan)

### 1.1.1 Model

#### Language

Classe de langage prennant en paramètre un corpus de la forme [[str]]

In [17]:
#from chatNLP.utils import Lang

In [18]:
class Lang:
    def __init__(self, corpus = None, base_tokens = ['SOS', 'EOS', 'UNK'], min_count = None):
        self.base_tokens = base_tokens
        self.initData(base_tokens)
        if    corpus is not None : self.addCorpus(corpus)
        if min_count is not None : self.removeRareWords(min_count)

        
    def initData(self, base_tokens) :
        self.word2index = {word : i for i, word in enumerate(base_tokens)}
        self.index2word = {i : word for i, word in enumerate(base_tokens)}
        self.word2count = {word : 0 for word in base_tokens}
        self.n_words = len(base_tokens)
        return
    
    def getIndex(self, word) :
        if    word in self.word2index : return self.word2index[word]
        elif 'UNK' in self.word2index : return self.word2index['UNK']
        return
        
    def addWord(self, word):
        '''Add a word to the language'''
        if word not in self.word2index:
            if word.strip() != '' :
                self.word2index[word] = self.n_words
                self.word2count[word] = 1
                self.index2word[self.n_words] = word
                self.n_words += 1
        else:
            self.word2count[word] += 1
        return 
            
    def addSentence(self, sentence):
        '''Add to the language all words of a sentence'''
        words = sentence if type(sentence) == list else nltk.word_tokenize(sentence)
        for word in words : self.addWord(word)          
        return
            
    def addCorpus(self, corpus):
        '''Add to the language all words contained into a corpus'''
        for text in corpus : self.addSentence(text)
        return 
                
    def removeRareWords(self, min_count):
        '''remove words appearing lesser than a min_count threshold'''
        kept_word2count = {word: count for word, count in self.word2count.items() if count >= min_count}
        self.initData(self.base_tokens)
        for word, count in kept_word2count.items(): 
            self.addWord(word)
            self.word2count[word] = kept_word2count[word]
        return

In [19]:
def saveLang(name, lang):
    with open(path_to_NLP + '\\saves\\lang\\' + name + '.file', 'wb') as fil :
        pickle.dump(lang, fil)
    return

def importLang(name):
    with open(path_to_NLP + '\\saves\\lang\\' + name + '.file', 'rb') as fil :
        lang = pickle.load(fil)
    return lang

In [20]:
lang = Lang(corpus, base_tokens = ['SOS', 'EOS', 'UNK'])
print("Mots comptés avant : {}".format(lang.n_words))
lang.removeRareWords(min_count = 4)
print("Mots comptés après : {}".format(lang.n_words))

Mots comptés avant : 8088
Mots comptés après : 4066


In [21]:
#saveLang(name = 'DL4NLP_I1', lang = lang)
#lang = importLang(name = 'DL4NLP_I1')

#### Comparaison avec un vocabulaire de référence

In [22]:
#taken from https://medium.com/@martinpella/how-to-use-pre-trained-word-embeddings-in-pytorch-71ca59249f76

# --------------------- comparison with Glove vocab ------------------------
def vocabGlove(name) :
    words = []
    path = path_to_NLP + '\\vectors\\' + name 
    with open(path + '.txt', 'rb') as f:
        for l in f:
            line = l.decode().split()
            word = line[0]
            words.append(word)
    return words

def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2))

def comparaison(lang) :
    vocab_lang = list(lang.word2index.keys())
    intersect_glove = intersection(vocab_glove, vocab_lang)
    reste_glove = np.setdiff1d(vocab_lang, intersect_glove)
    printComparaison('glove', vocab_lang, intersect_glove, reste_glove)
    return intersect_glove, reste_glove

def printComparaison(nom, vocab_lang, intersect, reste) :
    print('proportion de mots du langage appartenants à {}  {:.2f} % \nproportion de mots du langage ny appartenant pas     {:.2f} %'.format(nom, len(intersect)*100/len(vocab_lang),len(reste)*100/len(vocab_lang) ) )


# --------------------- detect missing spaces ------------------------
def checkWhetherBroken(vocab, clean_vocab) :
    exit = {}
    for word in vocab :
        exit[word] = True if word in clean_vocab else False
    return exit

def checkMissingSpaces(word, clean_vocab) :
    for word2 in clean_vocab :
        if word.startswith(word2) :
            rest = word.replace(word2, '')
            if rest in clean_vocab :
                return word2 + ' ' + rest
    return word

In [23]:
vocab_glove = vocabGlove('glove.6B.200d')

In [24]:
words_glove, reste_glove = comparaison(lang)

proportion de mots du langage appartenants à glove  85.76 % 
proportion de mots du langage ny appartenant pas     14.24 %


#### Word2Vec model

[Back to top](#plan)

In [12]:
#from chatNLP.models.Word_Embedding import Word2Vec as myWord2Vec

In [25]:
class myWord2Vec(nn.Module) :
    def __init__(self, lang, T = 100):
        super(myWord2Vec, self).__init__()
        self.lang = lang
        if type(T) == int :
            self.embedding = nn.Embedding(lang.n_words, T)  
        else :
            self.embedding = nn.Embedding(T.shape[0], T.shape[1])
            self.embedding.weight = nn.Parameter(torch.FloatTensor(T))
            
        self.output_dim = self.lookupTable().shape[1]
        self.sims = None
        
    def lookupTable(self) :
        return self.embedding.weight.cpu().detach().numpy()
        
    def computeSimilarities(self) :
        T = normalize(self.lookupTable(), norm = 'l2', axis = 1)
        self.sims = np.matmul(T, T.transpose())
        return

    def most_similar(self, word, bound = 10) :
        if word not in self.lang.word2index : return
        if self.sims is None : self.computeSimilarities()
        index = self.lang.word2index[word]
        coefs = self.sims[index]
        indices = coefs.argsort()[-bound -1 :-1]
        output = [(self.lang.index2word[i], coefs[i]) for i in reversed(indices)]
        return output
    
    def wv(self, word) :
        return self.lookupTable()[self.lang.getIndex(word)]
    
    def addWord(self, word, vector = None) :
        self.lang.addWord(word)
        T = self.lookupTable()
        v = np.random.rand(1, T.shape[1]) if vector is None else vector
        updated_T = np.concatenate((T, v), axis = 0)
        self.embedding = nn.Embedding(updated_T.shape[0], updated_T.shape[1])
        self.embedding.weight = nn.Parameter(torch.FloatTensor(updated_T))
        return
    
    def freeze(self) :
        for param in self.embedding.parameters() : param.requires_grad = False
        return self
    
    def unfreeze(self) :
        for param in self.embedding.parameters() : param.requires_grad = True
        return self
    
    def forward(self, words, device = None) :
        '''Transforms a list of n words into a torch.FloatTensor of size (1, n, emb_dim)'''
        indices  = [self.lang.getIndex(w) for w in words]
        indices  = [[i for i in indices if i is not None]]
        variable = Variable(torch.LongTensor(indices)) # size = (1, n)
        if device is not None : variable = variable.to(device)
        tensor   = self.embedding(variable)            # size = (1, n, emb_dim)
        return tensor

#### Word2Vec Shell

[Back to top](#plan)

Shell acting as a wrapper around the Word2Vec model, implementing :

- The layers suited for the training objective
- The methods for all optimization steps
- The methods for generating the data suitable for the optimization process

In [14]:
#from chatNLP.models.Word_Embedding import Word2VecShell

In [28]:
class Word2VecShell(nn.Module):
    '''Word2Vec model :
        - sg = 0 yields CBOW training procedure
        - sg = 1 yields Skip-Gram training procedure
    '''
    def __init__(self, word2vec, device, sg = 0, context_size = 5, hidden_dim = 150, 
                 criterion = nn.NLLLoss(size_average = False), optimizer = optim.SGD):
        super(Word2VecShell, self).__init__()
        self.device = device
        
        # core of Word2Vec
        self.word2vec = word2vec
        
        # training layers
        self.input_n_words  = (2 * context_size if sg == 0 else 1)
        self.output_n_words = (1 if sg == 0 else 2 * context_size)
        self.linear_1  = nn.Linear(self.input_n_words * word2vec.embedding.weight.size(1), self.output_n_words * hidden_dim)
        self.linear_2  = nn.Linear(hidden_dim, lang.n_words)
        
        # training tools
        self.sg = sg
        self.criterion = criterion
        self.optimizer = optimizer
        
        # load to device
        self.to(device)
        
    def forward(self, batch):
        '''Transforms a batch of Ngrams of size (batch_size, input_n_words)
           Into log probabilities of size (batch_size, lang.n_words, output_n_words)
           '''
        batch = batch.to(self.device)                 # size = (batch_size, self.input_n_words)
        embed = self.word2vec.embedding(batch)        # size = (batch_size, self.input_n_words, embedding_dim)
        embed = embed.view((batch.size(0), -1))       # size = (batch_size, self.input_n_words * embedding_dim)
        out = self.linear_1(embed)                    # size = (batch_size, self.output_n_words * hidden_dim) 
        out = out.view((batch.size(0),self.output_n_words, -1))
        out = F.relu(out)                             # size = (batch_size, self.output_n_words, hidden_dim)                                         
        out = self.linear_2(out)                      # size = (batch_size, self.output_n_words, lang.n_words)
        out = torch.transpose(out, 1, 2)              # size = (batch_size, lang.n_words, self.output_n_words)
        log_probs = F.log_softmax(out, dim = 1)       # size = (batch_size, lang.n_words, self.output_n_words)
        return log_probs
    
    def generatePackedNgrams(self, corpus, context_size = 5, batch_size = 32, seed = 42) :
        # generate Ngrams
        data = []
        for text in corpus :
            text = [w for w in text if w in self.word2vec.lang.word2index]
            text = ['SOS' for i in range(context_size)] + text + ['EOS' for i in range(context_size)]
            for i in range(context_size, len(text) - context_size):
                context = text[i-context_size : i] + text[i+1 : i+context_size+1]
                word = text[i]
                data.append([word, context])
        # pack Ngrams into mini_batches
        random.seed(seed)
        random.shuffle(data)
        packed_data = []
        for i in range(0, len(data), batch_size):
            pack0 = [el[0] for el in data[i:i + batch_size]]
            pack0 = [[self.word2vec.lang.getIndex(w)] for w in pack0]
            pack0 = Variable(torch.LongTensor(pack0)) # size = (batch_size, 1)
            pack1 = [el[1] for el in data[i:i + batch_size]]
            pack1 = [[self.word2vec.lang.getIndex(w) for w in context] for context in pack1]
            pack1 = Variable(torch.LongTensor(pack1)) # size = (batch_size, 2*context_size)   
            if   self.sg == 1 : packed_data.append([pack0, pack1])
            elif self.sg == 0 : packed_data.append([pack1, pack0])
            else :
                print('A problem occured')
                pass
        return packed_data
    
    def train(self, ngrams, iters = None, epochs = None, lr = 0.025, random_state = 42,
              print_every = 10, compute_accuracy = False):
        """Performs training over a given dataset and along a specified amount of loop
        s"""
        def asMinutes(s):
            m = math.floor(s / 60)
            s -= m * 60
            return '%dm %ds' % (m, s)

        def timeSince(since, percent):
            now = time.time()
            s = now - since
            rs = s/percent - s
            return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

        def computeAccuracy(log_probs, targets) :
            accuracy = 0
            for i in range(targets.size(0)) :
                for j in range(targets.size(1)) :
                    topv, topi = log_probs[i, :, j].data.topk(1) 
                    ni = topi[0][0]
                    if ni == targets[i, j].data[0] : accuracy += 1
            return (accuracy * 100) / (targets.size(0) * targets.size(1))

        def printScores(start, iter, iters, tot_loss, tot_loss_words, print_every, compute_accuracy) :
            avg_loss = tot_loss / print_every
            avg_loss_words = tot_loss_words / print_every
            if compute_accuracy : print(timeSince(start, iter / iters) + ' ({} {}%) loss : {:.3f}  accuracy : {:.1f} %'.format(iter, int(iter / iters * 100), avg_loss, avg_loss_words))
            else                : print(timeSince(start, iter / iters) + ' ({} {}%) loss : {:.3f}                     '.format(iter, int(iter / iters * 100), avg_loss))
            return 0, 0

        def trainLoop(couple, optimizer, compute_accuracy = False):
            """Performs a training loop, with forward pass and backward pass for gradient optimisation."""
            optimizer.zero_grad()
            self.zero_grad()
            log_probs = self(couple[0])           # size = (batch_size, agent.output_n_words, agent.lang.n_words)
            targets   = couple[1].to(self.device) # size = (batch_size, agent.output_n_words)
            loss      = self.criterion(log_probs, targets)
            loss.backward()
            optimizer.step() 
            accuracy = computeAccuracy(log_probs, targets) if compute_accuracy else 0
            return float(loss.data[0] / (targets.size(0) * targets.size(1))), accuracy
        
        # --- main ---
        np.random.seed(random_state)
        start = time.time()
        optimizer = self.optimizer([param for param in self.parameters() if param.requires_grad == True], lr = lr)
        tot_loss = 0  
        tot_loss_words = 0
        if epochs is None :
            for iter in range(1, iters + 1):
                couple = random.choice(ngrams)
                loss, loss_words = trainLoop(couple, optimizer, compute_accuracy)
                tot_loss += loss
                tot_loss_words += loss_words      
                if iter % print_every == 0 : 
                    tot_loss, tot_loss_words = printScores(start, iter, iters, tot_loss, tot_loss_words, print_every, compute_accuracy)
        else :
            iter = 0
            iters = len(ngrams) * epochs
            for epoch in range(1, epochs + 1):
                print('epoch ' + str(epoch))
                np.random.shuffle(ngrams)
                for couple in ngrams :
                    loss, loss_words = trainLoop(couple, optimizer, compute_accuracy)
                    tot_loss += loss
                    tot_loss_words += loss_words 
                    iter += 1
                    if iter % print_every == 0 : 
                        tot_loss, tot_loss_words = printScores(start, iter, iters, tot_loss, tot_loss_words, print_every, compute_accuracy)
        return

### 1.1.2 Training with CBOW objective

[Back to top](#plan)


Model

In [29]:
lang = Lang(corpus, base_tokens = ['SOS', 'EOS', 'UNK'], min_count = 4)

In [32]:
word2vec = myWord2Vec(lang, T = 75)
cbow = Word2VecShell(word2vec, device, sg = 0, context_size = 5, hidden_dim = 150)
print('cbow.word2vec = word2vec :', cbow.word2vec == word2vec)

cbow.word2vec = word2vec : True


Data

In [33]:
Ngrams = cbow.generatePackedNgrams(corpus, context_size = 5, batch_size = 32, seed = 42)

Training

The training methods allows to display accuracy over predicted target words. However, since the underlying computation is quite time consuming, we display accuracy only at the begining of training, and a few times periodically along the training process.

In [34]:
cbow.train(Ngrams, iters = 100, lr = 0.005, print_every = 100, compute_accuracy = True)

for alpha in [0.005, 0.001, 0.0005, 0.00025, 0.0001] : 
    cbow.train(Ngrams, epochs = 3,  lr = alpha, print_every = 100)
    cbow.train(Ngrams, iters = 100, lr = alpha, print_every = 100, compute_accuracy = True)

0m 2s (- 0m 0s) (100 100%) loss : 7.290  accuracy : 8.8 %
epoch 1
0m 0s (- 3m 57s) (100 0%) loss : 6.148                     
0m 1s (- 3m 50s) (200 0%) loss : 5.900                     
0m 1s (- 3m 46s) (300 0%) loss : 5.685                     
0m 2s (- 3m 44s) (400 0%) loss : 5.545                     
0m 2s (- 3m 42s) (500 1%) loss : 5.368                     
0m 3s (- 3m 41s) (600 1%) loss : 5.343                     
0m 3s (- 3m 41s) (700 1%) loss : 5.160                     
0m 4s (- 3m 40s) (800 1%) loss : 4.979                     
0m 4s (- 3m 39s) (900 2%) loss : 5.100                     
0m 5s (- 3m 39s) (1000 2%) loss : 4.979                     
0m 5s (- 3m 38s) (1100 2%) loss : 4.824                     
0m 6s (- 3m 37s) (1200 2%) loss : 4.881                     
0m 6s (- 3m 37s) (1300 3%) loss : 4.836                     
0m 7s (- 3m 36s) (1400 3%) loss : 4.726                     
0m 7s (- 3m 35s) (1500 3%) loss : 4.558                     
0m 8s (- 3m 35s) (1600 3%) l

1m 10s (- 2m 35s) (13200 31%) loss : 3.223                     
1m 10s (- 2m 34s) (13300 31%) loss : 3.394                     
1m 11s (- 2m 34s) (13400 31%) loss : 3.230                     
1m 11s (- 2m 33s) (13500 31%) loss : 3.258                     
1m 12s (- 2m 33s) (13600 31%) loss : 3.288                     
1m 12s (- 2m 32s) (13700 32%) loss : 3.251                     
1m 13s (- 2m 32s) (13800 32%) loss : 3.300                     
1m 13s (- 2m 31s) (13900 32%) loss : 3.204                     
1m 14s (- 2m 31s) (14000 32%) loss : 3.230                     
1m 14s (- 2m 30s) (14100 33%) loss : 3.216                     
epoch 2
1m 15s (- 2m 30s) (14200 33%) loss : 3.196                     
1m 15s (- 2m 29s) (14300 33%) loss : 2.827                     
1m 16s (- 2m 28s) (14400 33%) loss : 2.954                     
1m 16s (- 2m 28s) (14500 34%) loss : 2.853                     
1m 17s (- 2m 27s) (14600 34%) loss : 2.777                     
1m 17s (- 2m 27s) (14700 34%) lo

2m 17s (- 1m 26s) (26100 61%) loss : 2.874                     
2m 18s (- 1m 26s) (26200 61%) loss : 2.897                     
2m 18s (- 1m 25s) (26300 61%) loss : 2.941                     
2m 19s (- 1m 25s) (26400 62%) loss : 2.842                     
2m 19s (- 1m 24s) (26500 62%) loss : 2.771                     
2m 20s (- 1m 23s) (26600 62%) loss : 2.833                     
2m 21s (- 1m 23s) (26700 62%) loss : 2.780                     
2m 21s (- 1m 22s) (26800 63%) loss : 2.910                     
2m 22s (- 1m 22s) (26900 63%) loss : 2.894                     
2m 22s (- 1m 21s) (27000 63%) loss : 2.725                     
2m 23s (- 1m 21s) (27100 63%) loss : 2.917                     
2m 23s (- 1m 20s) (27200 63%) loss : 2.892                     
2m 24s (- 1m 20s) (27300 64%) loss : 2.908                     
2m 24s (- 1m 19s) (27400 64%) loss : 2.899                     
2m 25s (- 1m 19s) (27500 64%) loss : 2.860                     
2m 25s (- 1m 18s) (27600 64%) loss : 2.8

3m 26s (- 0m 18s) (39000 91%) loss : 2.594                     
3m 26s (- 0m 17s) (39100 91%) loss : 2.621                     
3m 27s (- 0m 17s) (39200 92%) loss : 2.650                     
3m 27s (- 0m 16s) (39300 92%) loss : 2.598                     
3m 28s (- 0m 16s) (39400 92%) loss : 2.655                     
3m 28s (- 0m 15s) (39500 92%) loss : 2.553                     
3m 29s (- 0m 15s) (39600 93%) loss : 2.520                     
3m 29s (- 0m 14s) (39700 93%) loss : 2.618                     
3m 30s (- 0m 14s) (39800 93%) loss : 2.607                     
3m 30s (- 0m 13s) (39900 93%) loss : 2.676                     
3m 31s (- 0m 13s) (40000 94%) loss : 2.538                     
3m 31s (- 0m 12s) (40100 94%) loss : 2.633                     
3m 32s (- 0m 12s) (40200 94%) loss : 2.588                     
3m 32s (- 0m 11s) (40300 94%) loss : 2.678                     
3m 33s (- 0m 11s) (40400 95%) loss : 2.554                     
3m 33s (- 0m 10s) (40500 95%) loss : 2.5

0m 51s (- 2m 55s) (9600 22%) loss : 1.897                     
0m 51s (- 2m 55s) (9700 22%) loss : 1.961                     
0m 52s (- 2m 54s) (9800 23%) loss : 1.896                     
0m 52s (- 2m 54s) (9900 23%) loss : 1.857                     
0m 53s (- 2m 53s) (10000 23%) loss : 1.870                     
0m 53s (- 2m 52s) (10100 23%) loss : 1.904                     
0m 54s (- 2m 52s) (10200 23%) loss : 1.852                     
0m 54s (- 2m 51s) (10300 24%) loss : 1.881                     
0m 55s (- 2m 51s) (10400 24%) loss : 1.865                     
0m 56s (- 2m 50s) (10500 24%) loss : 1.935                     
0m 56s (- 2m 50s) (10600 24%) loss : 1.853                     
0m 57s (- 2m 50s) (10700 25%) loss : 1.844                     
0m 57s (- 2m 49s) (10800 25%) loss : 1.898                     
0m 58s (- 2m 49s) (10900 25%) loss : 1.855                     
0m 58s (- 2m 48s) (11000 25%) loss : 1.847                     
0m 59s (- 2m 47s) (11100 26%) loss : 1.835  

...

2m 30s (- 1m 16s) (28100 66%) loss : 1.485                     
2m 30s (- 1m 16s) (28200 66%) loss : 1.478                     
2m 31s (- 1m 15s) (28300 66%) loss : 1.488                     
epoch 3
2m 31s (- 1m 15s) (28400 66%) loss : 1.384                     
2m 32s (- 1m 14s) (28500 67%) loss : 1.405                     
2m 32s (- 1m 14s) (28600 67%) loss : 1.413                     
2m 33s (- 1m 13s) (28700 67%) loss : 1.417                     
2m 33s (- 1m 13s) (28800 67%) loss : 1.379                     
2m 34s (- 1m 12s) (28900 67%) loss : 1.344                     
2m 34s (- 1m 12s) (29000 68%) loss : 1.375                     
2m 35s (- 1m 11s) (29100 68%) loss : 1.444                     
2m 35s (- 1m 10s) (29200 68%) loss : 1.433                     
2m 36s (- 1m 10s) (29300 68%) loss : 1.394                     
2m 36s (- 1m 9s) (29400 69%) loss : 1.424                     
2m 37s (- 1m 9s) (29500 69%) loss : 1.352                     
2m 37s (- 1m 8s) (29600 69%) loss 

3m 33s (- 0m 7s) (41000 96%) loss : 1.439                     
3m 33s (- 0m 7s) (41100 96%) loss : 1.427                     
3m 34s (- 0m 6s) (41200 96%) loss : 1.511                     
3m 34s (- 0m 6s) (41300 97%) loss : 1.484                     
3m 34s (- 0m 5s) (41400 97%) loss : 1.436                     
3m 35s (- 0m 5s) (41500 97%) loss : 1.457                     
3m 35s (- 0m 4s) (41600 97%) loss : 1.454                     
3m 36s (- 0m 4s) (41700 98%) loss : 1.504                     
3m 36s (- 0m 3s) (41800 98%) loss : 1.475                     
3m 37s (- 0m 3s) (41900 98%) loss : 1.516                     
3m 37s (- 0m 2s) (42000 98%) loss : 1.485                     
3m 38s (- 0m 2s) (42100 99%) loss : 1.422                     
3m 38s (- 0m 1s) (42200 99%) loss : 1.408                     
3m 39s (- 0m 1s) (42300 99%) loss : 1.448                     
3m 39s (- 0m 0s) (42400 99%) loss : 1.473                     
3m 40s (- 0m 0s) (42500 99%) loss : 1.463              

0m 54s (- 2m 25s) (11600 27%) loss : 1.328                     
0m 55s (- 2m 25s) (11700 27%) loss : 1.420                     
0m 55s (- 2m 25s) (11800 27%) loss : 1.396                     
0m 56s (- 2m 24s) (11900 27%) loss : 1.442                     
0m 56s (- 2m 24s) (12000 28%) loss : 1.397                     
0m 57s (- 2m 23s) (12100 28%) loss : 1.487                     
0m 57s (- 2m 23s) (12200 28%) loss : 1.431                     
0m 58s (- 2m 22s) (12300 28%) loss : 1.410                     
0m 58s (- 2m 22s) (12400 29%) loss : 1.382                     
0m 59s (- 2m 21s) (12500 29%) loss : 1.477                     
0m 59s (- 2m 21s) (12600 29%) loss : 1.459                     
0m 59s (- 2m 20s) (12700 29%) loss : 1.404                     
1m 0s (- 2m 20s) (12800 30%) loss : 1.362                     
1m 0s (- 2m 19s) (12900 30%) loss : 1.407                     
1m 1s (- 2m 19s) (13000 30%) loss : 1.323                     
1m 1s (- 2m 18s) (13100 30%) loss : 1.452  

1m 55s (- 1m 25s) (24500 57%) loss : 1.366                     
1m 56s (- 1m 24s) (24600 57%) loss : 1.434                     
1m 56s (- 1m 24s) (24700 58%) loss : 1.380                     
1m 57s (- 1m 23s) (24800 58%) loss : 1.340                     
1m 57s (- 1m 23s) (24900 58%) loss : 1.351                     
1m 58s (- 1m 22s) (25000 58%) loss : 1.400                     
1m 58s (- 1m 22s) (25100 59%) loss : 1.409                     
1m 58s (- 1m 21s) (25200 59%) loss : 1.456                     
1m 59s (- 1m 21s) (25300 59%) loss : 1.383                     
1m 59s (- 1m 20s) (25400 59%) loss : 1.475                     
2m 0s (- 1m 20s) (25500 59%) loss : 1.420                     
2m 0s (- 1m 19s) (25600 60%) loss : 1.416                     
2m 1s (- 1m 19s) (25700 60%) loss : 1.409                     
2m 1s (- 1m 18s) (25800 60%) loss : 1.454                     
2m 2s (- 1m 18s) (25900 60%) loss : 1.365                     
2m 2s (- 1m 17s) (26000 61%) loss : 1.387    

2m 56s (- 0m 24s) (37400 87%) loss : 1.420                     
2m 57s (- 0m 23s) (37500 88%) loss : 1.321                     
2m 57s (- 0m 23s) (37600 88%) loss : 1.352                     
2m 58s (- 0m 22s) (37700 88%) loss : 1.378                     
2m 58s (- 0m 22s) (37800 88%) loss : 1.378                     
2m 59s (- 0m 21s) (37900 89%) loss : 1.422                     
2m 59s (- 0m 21s) (38000 89%) loss : 1.462                     
3m 0s (- 0m 20s) (38100 89%) loss : 1.371                     
3m 0s (- 0m 20s) (38200 89%) loss : 1.354                     
3m 1s (- 0m 19s) (38300 90%) loss : 1.441                     
3m 1s (- 0m 19s) (38400 90%) loss : 1.394                     
3m 1s (- 0m 18s) (38500 90%) loss : 1.352                     
3m 2s (- 0m 18s) (38600 90%) loss : 1.423                     
3m 2s (- 0m 17s) (38700 91%) loss : 1.331                     
3m 3s (- 0m 17s) (38800 91%) loss : 1.416                     
3m 3s (- 0m 17s) (38900 91%) loss : 1.436       

Evaluation

In [35]:
word2vec.most_similar(word = 'formaldehyde', bound = 10)

[('pasteurized', 0.37487525),
 ('establish', 0.35843933),
 ('sodium', 0.35818478),
 ('responders', 0.35638148),
 ('indicative', 0.33886254),
 ('protective', 0.3352614),
 ('order', 0.32748583),
 ('hydrochloride', 0.31852683),
 ('positively', 0.3115115),
 ('greatest', 0.3066505)]

Save & Load<br>

The lightweight word2vec model can be saved for further use, or alternatively the full shell wrapping the word2vec model can be saved for subsequent training.

In [36]:
# save
#torch.save(word2vec, path_to_NLP + '\\saves\\models\\DL4NLP_I1_cbow.pt')

# load
#word2vec = torch.load(path_to_NLP + '\\saves\\models\\DL4NLP_I1_cbow.pt')

### 1.1.3 Training with SkipGram objective

[Back to top](#plan)


Model

In [37]:
lang = Lang(corpus, base_tokens = ['SOS', 'EOS', 'UNK'], min_count = 4)

In [38]:
word2vec = myWord2Vec(lang, T = 75)
skipgram = Word2VecShell(word2vec, device, sg = 1, context_size = 5, hidden_dim = 150)
print('skipgram.word2vec = word2vec :', skipgram.word2vec == word2vec)

skipgram.word2vec = word2vec : True


Data

In [39]:
Ngrams = skipgram.generatePackedNgrams(corpus, context_size = 5, batch_size = 32, seed = 42)

Training

In [40]:
skipgram.train(Ngrams, iters = 100, lr = 0.005, print_every = 100, compute_accuracy = True)

for alpha in [0.005, 0.001, 0.0005] : 
    skipgram.train(Ngrams, epochs = 3,  lr = alpha, print_every = 100)
    skipgram.train(Ngrams, iters = 100, lr = alpha, print_every = 100, compute_accuracy = True)

0m 12s (- 0m 0s) (100 100%) loss : 6.662  accuracy : 7.8 %
epoch 1
0m 1s (- 11m 9s) (100 0%) loss : 6.199                     
0m 3s (- 11m 4s) (200 0%) loss : 6.103                     
0m 4s (- 11m 1s) (300 0%) loss : 6.045                     
0m 6s (- 10m 59s) (400 0%) loss : 6.030                     
0m 7s (- 10m 58s) (500 1%) loss : 6.035                     
0m 9s (- 10m 56s) (600 1%) loss : 5.978                     
0m 10s (- 10m 54s) (700 1%) loss : 5.958                     
0m 12s (- 10m 52s) (800 1%) loss : 5.923                     
0m 14s (- 10m 51s) (900 2%) loss : 5.963                     
0m 15s (- 10m 49s) (1000 2%) loss : 5.915                     
0m 17s (- 10m 47s) (1100 2%) loss : 5.873                     
0m 18s (- 10m 46s) (1200 2%) loss : 5.904                     
0m 20s (- 10m 44s) (1300 3%) loss : 5.873                     
0m 21s (- 10m 42s) (1400 3%) loss : 5.892                     
0m 23s (- 10m 41s) (1500 3%) loss : 5.850                     
0m 25s

3m 40s (- 8m 20s) (13000 30%) loss : 5.536                     
3m 41s (- 8m 18s) (13100 30%) loss : 5.534                     
3m 43s (- 8m 16s) (13200 31%) loss : 5.480                     
3m 45s (- 8m 14s) (13300 31%) loss : 5.512                     
3m 46s (- 8m 12s) (13400 31%) loss : 5.491                     
3m 48s (- 8m 10s) (13500 31%) loss : 5.473                     
3m 49s (- 8m 8s) (13600 31%) loss : 5.513                     
3m 51s (- 8m 6s) (13700 32%) loss : 5.508                     
3m 52s (- 8m 4s) (13800 32%) loss : 5.480                     
3m 54s (- 8m 2s) (13900 32%) loss : 5.502                     
3m 56s (- 8m 0s) (14000 32%) loss : 5.479                     
3m 57s (- 7m 58s) (14100 33%) loss : 5.460                     
epoch 2
3m 59s (- 7m 56s) (14200 33%) loss : 5.490                     
4m 0s (- 7m 54s) (14300 33%) loss : 5.434                     
4m 2s (- 7m 52s) (14400 33%) loss : 5.482                     
4m 3s (- 7m 50s) (14500 34%) loss : 5.4

7m 2s (- 4m 30s) (25900 60%) loss : 5.348                     
7m 4s (- 4m 29s) (26000 61%) loss : 5.371                     
7m 5s (- 4m 27s) (26100 61%) loss : 5.403                     
7m 7s (- 4m 25s) (26200 61%) loss : 5.388                     
7m 8s (- 4m 24s) (26300 61%) loss : 5.412                     
7m 10s (- 4m 22s) (26400 62%) loss : 5.362                     
7m 11s (- 4m 20s) (26500 62%) loss : 5.367                     
7m 13s (- 4m 19s) (26600 62%) loss : 5.374                     
7m 15s (- 4m 17s) (26700 62%) loss : 5.339                     
7m 16s (- 4m 15s) (26800 63%) loss : 5.417                     
7m 18s (- 4m 14s) (26900 63%) loss : 5.417                     
7m 19s (- 4m 12s) (27000 63%) loss : 5.372                     
7m 21s (- 4m 10s) (27100 63%) loss : 5.411                     
7m 22s (- 4m 9s) (27200 63%) loss : 5.416                     
7m 24s (- 4m 7s) (27300 64%) loss : 5.419                     
7m 25s (- 4m 5s) (27400 64%) loss : 5.417      

10m 24s (- 0m 59s) (38800 91%) loss : 5.305                     
10m 25s (- 0m 57s) (38900 91%) loss : 5.354                     
10m 27s (- 0m 56s) (39000 91%) loss : 5.353                     
10m 28s (- 0m 54s) (39100 91%) loss : 5.302                     
10m 30s (- 0m 53s) (39200 92%) loss : 5.375                     
10m 31s (- 0m 51s) (39300 92%) loss : 5.322                     
10m 33s (- 0m 49s) (39400 92%) loss : 5.333                     
10m 35s (- 0m 48s) (39500 92%) loss : 5.338                     
10m 36s (- 0m 46s) (39600 93%) loss : 5.323                     
10m 38s (- 0m 45s) (39700 93%) loss : 5.331                     
10m 39s (- 0m 43s) (39800 93%) loss : 5.360                     
10m 41s (- 0m 41s) (39900 93%) loss : 5.320                     
10m 42s (- 0m 40s) (40000 94%) loss : 5.313                     
10m 44s (- 0m 38s) (40100 94%) loss : 5.336                     
10m 46s (- 0m 37s) (40200 94%) loss : 5.304                     
10m 47s (- 0m 35s) (40300

2m 23s (- 8m 40s) (9200 21%) loss : 5.213                     
2m 25s (- 8m 38s) (9300 21%) loss : 5.176                     
2m 26s (- 8m 37s) (9400 22%) loss : 5.196                     
2m 28s (- 8m 35s) (9500 22%) loss : 5.184                     
2m 29s (- 8m 34s) (9600 22%) loss : 5.211                     
2m 31s (- 8m 32s) (9700 22%) loss : 5.214                     
2m 33s (- 8m 30s) (9800 23%) loss : 5.209                     
2m 34s (- 8m 29s) (9900 23%) loss : 5.203                     
2m 36s (- 8m 27s) (10000 23%) loss : 5.186                     
2m 37s (- 8m 26s) (10100 23%) loss : 5.213                     
2m 39s (- 8m 24s) (10200 23%) loss : 5.197                     
2m 40s (- 8m 23s) (10300 24%) loss : 5.233                     
2m 42s (- 8m 21s) (10400 24%) loss : 5.222                     
2m 44s (- 8m 19s) (10500 24%) loss : 5.183                     
2m 45s (- 8m 18s) (10600 24%) loss : 5.184                     
2m 47s (- 8m 16s) (10700 25%) loss : 5.220      

6m 2s (- 5m 34s) (22100 51%) loss : 5.219                     
6m 4s (- 5m 33s) (22200 52%) loss : 5.178                     
6m 5s (- 5m 31s) (22300 52%) loss : 5.180                     
6m 7s (- 5m 29s) (22400 52%) loss : 5.171                     
6m 9s (- 5m 28s) (22500 52%) loss : 5.204                     
6m 10s (- 5m 26s) (22600 53%) loss : 5.225                     
6m 12s (- 5m 24s) (22700 53%) loss : 5.214                     
6m 13s (- 5m 22s) (22800 53%) loss : 5.187                     
6m 15s (- 5m 21s) (22900 53%) loss : 5.192                     
6m 16s (- 5m 19s) (23000 54%) loss : 5.201                     
6m 18s (- 5m 17s) (23100 54%) loss : 5.182                     
6m 20s (- 5m 16s) (23200 54%) loss : 5.187                     
6m 21s (- 5m 14s) (23300 54%) loss : 5.177                     
6m 23s (- 5m 12s) (23400 55%) loss : 5.193                     
6m 24s (- 5m 11s) (23500 55%) loss : 5.144                     
6m 26s (- 5m 9s) (23600 55%) loss : 5.173    

9m 24s (- 2m 1s) (35000 82%) loss : 5.168                     
9m 26s (- 1m 59s) (35100 82%) loss : 5.164                     
9m 28s (- 1m 57s) (35200 82%) loss : 5.182                     
9m 29s (- 1m 56s) (35300 83%) loss : 5.168                     
9m 31s (- 1m 54s) (35400 83%) loss : 5.159                     
9m 32s (- 1m 52s) (35500 83%) loss : 5.147                     
9m 34s (- 1m 51s) (35600 83%) loss : 5.178                     
9m 35s (- 1m 49s) (35700 83%) loss : 5.148                     
9m 37s (- 1m 48s) (35800 84%) loss : 5.185                     
9m 38s (- 1m 46s) (35900 84%) loss : 5.197                     
9m 40s (- 1m 44s) (36000 84%) loss : 5.161                     
9m 42s (- 1m 43s) (36100 84%) loss : 5.170                     
9m 43s (- 1m 41s) (36200 85%) loss : 5.186                     
9m 45s (- 1m 40s) (36300 85%) loss : 5.159                     
9m 46s (- 1m 38s) (36400 85%) loss : 5.174                     
9m 48s (- 1m 36s) (36500 85%) loss : 5.18

1m 22s (- 9m 40s) (5300 12%) loss : 5.128                     
1m 24s (- 9m 39s) (5400 12%) loss : 5.126                     
1m 25s (- 9m 37s) (5500 12%) loss : 5.130                     
1m 27s (- 9m 35s) (5600 13%) loss : 5.123                     
1m 28s (- 9m 34s) (5700 13%) loss : 5.096                     
1m 30s (- 9m 32s) (5800 13%) loss : 5.138                     
1m 32s (- 9m 31s) (5900 13%) loss : 5.110                     
1m 33s (- 9m 29s) (6000 14%) loss : 5.152                     
1m 35s (- 9m 28s) (6100 14%) loss : 5.087                     
1m 36s (- 9m 26s) (6200 14%) loss : 5.161                     
1m 38s (- 9m 25s) (6300 14%) loss : 5.131                     
1m 39s (- 9m 23s) (6400 15%) loss : 5.130                     
1m 41s (- 9m 21s) (6500 15%) loss : 5.162                     
1m 43s (- 9m 20s) (6600 15%) loss : 5.149                     
1m 44s (- 9m 18s) (6700 15%) loss : 5.125                     
1m 46s (- 9m 17s) (6800 15%) loss : 5.147              

4m 46s (- 6m 18s) (18300 43%) loss : 5.163                     
4m 47s (- 6m 16s) (18400 43%) loss : 5.162                     
4m 49s (- 6m 15s) (18500 43%) loss : 5.154                     
4m 50s (- 6m 13s) (18600 43%) loss : 5.130                     
4m 52s (- 6m 12s) (18700 43%) loss : 5.130                     
4m 53s (- 6m 10s) (18800 44%) loss : 5.170                     
4m 55s (- 6m 9s) (18900 44%) loss : 5.129                     
4m 57s (- 6m 7s) (19000 44%) loss : 5.125                     
4m 58s (- 6m 5s) (19100 44%) loss : 5.166                     
5m 0s (- 6m 4s) (19200 45%) loss : 5.147                     
5m 1s (- 6m 2s) (19300 45%) loss : 5.125                     
5m 3s (- 6m 1s) (19400 45%) loss : 5.154                     
5m 4s (- 5m 59s) (19500 45%) loss : 5.163                     
5m 6s (- 5m 58s) (19600 46%) loss : 5.144                     
5m 7s (- 5m 56s) (19700 46%) loss : 5.131                     
5m 9s (- 5m 54s) (19800 46%) loss : 5.167           

8m 7s (- 2m 56s) (31200 73%) loss : 5.146                     
8m 9s (- 2m 55s) (31300 73%) loss : 5.159                     
8m 11s (- 2m 53s) (31400 73%) loss : 5.082                     
8m 12s (- 2m 52s) (31500 74%) loss : 5.155                     
8m 14s (- 2m 50s) (31600 74%) loss : 5.084                     
8m 15s (- 2m 48s) (31700 74%) loss : 5.139                     
8m 17s (- 2m 47s) (31800 74%) loss : 5.091                     
8m 18s (- 2m 45s) (31900 75%) loss : 5.122                     
8m 20s (- 2m 44s) (32000 75%) loss : 5.114                     
8m 21s (- 2m 42s) (32100 75%) loss : 5.160                     
8m 23s (- 2m 41s) (32200 75%) loss : 5.104                     
8m 25s (- 2m 39s) (32300 75%) loss : 5.110                     
8m 26s (- 2m 38s) (32400 76%) loss : 5.136                     
8m 28s (- 2m 36s) (32500 76%) loss : 5.096                     
8m 29s (- 2m 34s) (32600 76%) loss : 5.103                     
8m 31s (- 2m 33s) (32700 76%) loss : 5.124

Evaluation

In [41]:
word2vec.most_similar(word = 'final', bound = 10)

[('fdnc1947', 0.37422585),
 ('vendor', 0.3686211),
 ('nitrate', 0.3527695),
 ('pairwise', 0.336586),
 ('phase', 0.33086002),
 ('hydride', 0.32060236),
 ('mineralizate', 0.31994864),
 ('kinetic', 0.31498104),
 ('filled', 0.3145452),
 ('daily', 0.30329823)]

Save & Load<br>

The lightweight word2vec model can be saved for further use, or alternatively the full shell wrapping the word2vec model can be saved for subsequent training.

In [42]:
# save
#torch.save(word2vec, path_to_NLP + '\\saves\\models\\DL4NLP_I1_skipgram.pt')

# load
#word2vec = torch.load(path_to_NLP + '\\saves\\models\\DL4NLP_I1_skipgram.pt')

<a id="gensim"></a>

## 1.2 Gensim Word2Vec

[Back to top](#plan)

Link : https://radimrehurek.com/gensim/models/word2vec.html<br>
Tutorials :

- https://cambridgespark.com/4046-2/
- https://rare-technologies.com/word2vec-tutorial/
- http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/

### 1.2.1 Model

In [43]:
from gensim.models import Word2Vec
import multiprocessing
from gensim.test.utils import datapath, get_tmpfile

### 1.2.2 Training with CBOW objective

[Back to top](#plan)

Model & Data & Training

In [44]:
cbow_gensim = Word2Vec(corpus, 
                       size = 75, 
                       window = 5, 
                       min_count = 4, 
                       negative = 15, 
                       iter = 50,
                       sg = 0,
                       workers = multiprocessing.cpu_count())

Evaluation

In [45]:
#help(cbow_gensim)
cbow_gensim.wv.most_similar('formaldehyde')

[('thiomersal', 0.6767041683197021),
 ('polysorbate', 0.6719549298286438),
 ('ovalbumin', 0.6334631443023682),
 ('moisture', 0.6150280833244324),
 ('2phenoxyethanol', 0.6061439514160156),
 ('phenoxyethanol', 0.6048438549041748),
 ('phosphorus', 0.5749061107635498),
 ('aluminium', 0.553789496421814),
 ('phenol', 0.5379729270935059),
 ('sucrose', 0.5111640691757202)]

Save & Load

The Gensim model can easily be saved & loaded :

In [46]:
# save
#file_name = get_tmpfile(path_to_NLP + "\\saves\\models\\DL4NLP_I1_cbow_gensim.model")
#cbow_gensim.save(file_name)

# load
#file_name = get_tmpfile(path_to_NLP + "\\saves\\models\\DL4NLP_I1_cbow_gensim.model")
#cbow_gensim = Word2Vec.load(file_name)

Alternatively it is direct to build a lightweight word2vec model out of a trained gensim model and then save & load it as done in previous section.

In [86]:
word2vec = myWord2Vec(lang = Lang(corpus = [list(cbow_gensim.wv.index2word)], base_tokens = []), T = cbow_gensim.wv.vectors)

In [88]:
word2vec.most_similar('formaldehyde')

[('polysorbate', 0.6989662),
 ('ovalbumin', 0.67078525),
 ('thiomersal', 0.6553794),
 ('phenol', 0.6534851),
 ('phosphorus', 0.6203261),
 ('phenoxyethanol', 0.61794806),
 ('moisture', 0.5869265),
 ('sucrose', 0.5719405),
 ('2phenoxyethanol', 0.5603489),
 ('aluminium', 0.535767)]

### 1.2.3 Training with SkipGram objective

[Back to top](#plan)

Model & Data & Training

In [47]:
skipgram_gensim = Word2Vec(corpus, 
                           size = 75, 
                           window = 5, 
                           min_count = 4, 
                           negative = 15, 
                           iter = 50,
                           sg = 1,
                           workers = multiprocessing.cpu_count())

Evaluation

In [48]:
#help(skipgram_gensim)
skipgram_gensim.wv.most_similar('formaldehyde')

[('thiomersal', 0.6575016975402832),
 ('ovalbumin', 0.6415812969207764),
 ('phenoxyethanol', 0.6408674716949463),
 ('phenol', 0.6294451951980591),
 ('hcho', 0.62266606092453),
 ('residual', 0.587759256362915),
 ('free', 0.5830410718917847),
 ('aluminium', 0.5764816999435425),
 ('polysorbate', 0.5677136182785034),
 ('triton', 0.5672757625579834)]

Save & Load

In [49]:
# save
#file_name = get_tmpfile(path_to_NLP + "\\saves\\models\\DL4NLP_I1_skipgram_gensim.model")
#skipgram_gensim.save(file_name)

# load
#file_name = get_tmpfile(path_to_NLP + "\\saves\\models\\DL4NLP_I1_skipgram_gensim.model")
#skipgram_gensim = Word2Vec.load(file_name)

<a id="sub_word_level"></a>


# 2 Subword-level Embedding
***

<a id="fastText"></a>

## 2.1 FastText Subword-level Embedding Model

[Back to top](#plan)


We consider the Gensim implementation of FastText, based on the CBOW training objective.<br>
Tutorial : [Gensim FastText](https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/FastText_Tutorial.ipynb)<br>
Link to the original paper : [Enriching Word Vectors with Subword Information](https://arxiv.org/pdf/1607.04606.pdf).

### 2.1.1 Model

In [50]:
from gensim.models.fasttext import FastText as FT_gensim
from gensim.test.utils import datapath, get_tmpfile

### 2.1.2 Training with CBOW objective

[Back to top](#plan)

Model & Data

In [51]:
cbow_fastText_gensim = FT_gensim(size = 75, 
                                 window = 5, 
                                 min_count = 4, 
                                 negative = 15,
                                 sg = 0)

In [52]:
cbow_fastText_gensim.build_vocab(corpus)

Training

In [53]:
cbow_fastText_gensim.train(sentences = corpus, 
                           epochs = 50,
                           total_examples = cbow_fastText_gensim.corpus_count)

Evaluation

In [54]:
cbow_fastText_gensim.wv.most_similar('formaldehyde')

[('glutaraldehyde', 0.7824963331222534),
 ('thiomersal', 0.6987553834915161),
 ('phenoxyethanol', 0.66783607006073),
 ('ovalbumin', 0.6669738292694092),
 ('polysorbate', 0.6604278087615967),
 ('2phenoxyethanol', 0.6562307476997375),
 ('formal', 0.6519432067871094),
 ('acetaldehyde', 0.6417372226715088),
 ('phenoxy', 0.6388593316078186),
 ('phenol', 0.6205344200134277)]

Save & Load

In [55]:
# save
#file_name = get_tmpfile(path_to_NLP + "\\saves\\models\\DL4NLP_I1_cbow_fasttext.model")
#cbow_fastText_gensim.save(file_name)

# load
#file_name = get_tmpfile(path_to_NLP + "\\saves\\models\\DL4NLP_I1_cbow_fasttext.model")
#cbow_fastText_gensim = FT_gensim.load(file_name)

Alternatively it is direct to build a lightweight word2vec model out of a trained gensim model and then save & load it as done in previous section.

In [94]:
word2vec = myWord2Vec(lang = Lang(corpus = [list(cbow_fastText_gensim.wv.index2word)], base_tokens = []), T = cbow_fastText_gensim.wv.vectors)

However, the main advantage FastText offers is the possibility to get an embedding vector out of **any word**, and in fact any string thanks to the character-ngrams embedding trick :

In [ ]:
cbow_fastText_gensim['HelloWorld']

Nonetheless, it can be interesting to load the look-up word vectors table into a lightweight word2vec module, as it allows to further optimize this table for any specific downstream task performed by a larger PyTorch model.

### 2.1.3 Training with SkipGram objective

[Back to top](#plan)

Model & Data

In [56]:
fastText_gensim = FT_gensim(size = 75, 
                           window = 5, 
                           min_count = 4, 
                           negative = 15,
                           sg = 1)

In [57]:
fastText_gensim.build_vocab(corpus)

Training

In [58]:
fastText_gensim.train(sentences = corpus, 
                      epochs = 50,
                      total_examples = fastText_gensim.corpus_count)

Evaluation

In [59]:
fastText_gensim.wv.most_similar('formaldehyde')

[('phenoxyethanol', 0.7352535724639893),
 ('ovalbumin', 0.7251032590866089),
 ('2phenoxyethanol', 0.710415244102478),
 ('hcho', 0.6905250549316406),
 ('polysorbate', 0.684985876083374),
 ('residual', 0.639625608921051),
 ('thiomersal', 0.6269700527191162),
 ('acetyl', 0.6253007650375366),
 ('phosphorus', 0.6174876093864441),
 ('phenol', 0.6120678186416626)]

Save & Load

In [60]:
# save
#file_name = get_tmpfile(path_to_NLP + "\\saves\\models\\DL4NLP_I1_fasttext.model")
#fastText_gensim.save(file_name)

# load
#file_name = get_tmpfile(path_to_NLP + "\\saves\\models\\DL4NLP_I1_fasttext.model")
#fastText_gensim = FT_gensim.load(file_name)

In [ ]:
fastText_gensim[['13', 'to']]